In [6]:
import numpy as np
import pandas as pd
import tensorflow as tf
import os
import matplotlib.pyplot as plt
import matplotlib
import librosa
import wave
import keras
from keras.callbacks import LearningRateScheduler
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import *
from keras.optimizers import rmsprop
import seaborn as sn
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
import time
import sys
import glob
import math
from scipy.fftpack import fft
from scipy.fftpack.realtransforms import dct
from scipy.signal import lfilter, hamming
from imblearn.over_sampling import SMOTE
from keras.optimizers import SGD
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.utils import class_weight

eps = 0.00000001
print ("setup complete")

Using TensorFlow backend.


setup complete


In [ ]:
def extract_mfcc(wav_file_name):
    '''This function extracts mfcc features and obtain the mean of each dimension
    Input : path_to_wav_file
    Output: mfcc_features'''
    y, sr = librosa.load(wav_file_name, res_type='kaiser_best', duration=3, offset=0.5)
#     trimmed_data = np.zeros((160, 20))
    mfccs = np.mean(librosa.feature.mfcc(y=y, sr=sr, n_mfcc=40).T,axis=0)
#     data = np.array(librosa.feature.mfcc(y = y, sr = sr, n_mfcc=40).T)
#     if data.shape[0] <= 160:
#         trimmed_data[:data.shape[0],0:] = data[:,0:]
#     else:
#         trimmed_data[0:,0:] = data[0:160,0:]
    return mfccs

In [ ]:
def genderClassification(item):
    global gender_list
    if int(item[18:-4]) % 2 == 0:
        gender_list = 1  # female
    if int(item[18:-4]) % 2 == 1: 
        gender_list = 0  # male 
    return gender_list

In [ ]:
def genderClassification_another(item):
    global gender_list
    if item[0] == 'f':
        gender_list = 1  # female
    if item[0] == 'm': 
        gender_list = 0  # male 
    return gender_list

In [ ]:
def classification(item):
    global feeling_list
    if item[11:14] == 'ANG':
        feeling_list = 4  
    if item[11:14] == 'DIS': 
        feeling_list = 6
    if item[11:14] == 'FEA':
        feeling_list = 5
    if item[11:14] == 'HAP':
        feeling_list = 2
#     if item[11:14] == 'NEU':
#         feeling_list = 0
    if item[11:14] == 'SAD':
        feeling_list = 3
    return feeling_list

In [ ]:
# ### although, we load the data here, it is not used in training or validation
# for dirname, _, filenames in os.walk('/kaggle/input/audiowav-cremad/AudioWAV_cremad/'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))
root_dir = "/kaggle/input/audiowav-cremad/AudioWAV_cremad/" 

without_gender_cremad_only_data = [] ###stores the mfcc data
without_gender_cremad_only_labels = [] ###stores the labels
for subdirs, dirs, files in os.walk(root_dir):
    for file in files:
        if file[11:14] == 'NEU':
            continue
        else:
            y, sr = librosa.load(os.path.join(subdirs,file), res_type='kaiser_best', 
                                 duration=librosa.get_duration(filename=os.path.join(subdirs,file)), offset=0.25)
            mfccs = np.mean(librosa.feature.mfcc(y=y, sr=sr, n_mfcc=40).T,axis=0)
            without_gender_cremad_only_data.append(mfccs)
            without_gender_cremad_only_labels.append(classification(file))

In [ ]:
#### convert data to array and make labels categorical
without_gender_cremad_only_data_array = np.array(without_gender_cremad_only_data)
without_gender_cremad_only_labels_array = np.array(without_gender_cremad_only_labels)
without_gender_cremad_only_data_array.shape

In [ ]:
# for dirname, _, filenames in os.walk('/kaggle/input/ravdessaudiofilessfromvideo/ravdess_audio_files_from_video/'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

### extract audio data from AV RAVDESS data
root_dir = "/kaggle/input/ravdessaudiofilessfromvideo/ravdess_audio_files_from_video/" 

without_gender_audio_only_data = []  # stores the mfcc data
without_gender_audio_only_labels = []  # stores the labels
for subdirs, dirs, files in os.walk(root_dir):
    for file in files:
        if int(file[7:8]) == 1:
                continue
        if int(file[7:8]) == 2:
            continue
        else:
            y, sr = librosa.load(os.path.join(subdirs, file), res_type='kaiser_best', 
                                 duration=librosa.get_duration(filename=os.path.join(subdirs,file)), offset=0.5)
            mfccs = np.mean(librosa.feature.mfcc(y=y, sr=sr, n_mfcc=40).T, axis=0)
            without_gender_audio_only_data.append(mfccs)
            without_gender_audio_only_labels.append(int(file[7:8]) - 1)

In [ ]:
#### convert data to array and make labels categorical
without_gender_audio_only_data_array = np.array(without_gender_audio_only_data)
without_gender_audio_only_labels_array = np.array(without_gender_audio_only_labels)
without_gender_audio_only_data_array.shape
without_gender_audio_only_labels_array.shape

In [ ]:
##### load data from savee dataset
#### although, we load the data here, it is not used in training or validation
# for dirname, _, filenames in os.walk('/kaggle/input/savee-database/audiodata/AudioData/'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))
root_dir = "/kaggle/input/savee-database/audiodata/AudioData/"
# root_dir = "../input/audio_speech_actors_01-24/"
without_gender_savee_data = []
without_gender_savee_labels = []
for actor_dir in sorted(os.listdir(root_dir)):
    if actor_dir[-4:] == ".txt":
        continue
    for file_name in os.listdir(os.path.join(root_dir, actor_dir)):
        if file_name[0] == "c":
            continue
        if file_name[0] == "n":
            continue
        wav_file_name = os.path.join(root_dir, actor_dir, file_name)
        without_gender_savee_data.append(extract_mfcc(wav_file_name))
        if file_name[0] == "a":
            without_gender_savee_labels.append(4)
        if file_name[0] == "d":
            without_gender_savee_labels.append(6)
        if file_name[0] == "f":
            without_gender_savee_labels.append(5)
        if file_name[0] == "h":
            without_gender_savee_labels.append(2)
        if file_name[:2] == "sa":
            without_gender_savee_labels.append(3)
        if file_name[:2] == "su":
            without_gender_savee_labels.append(7)

In [ ]:
#### convert data to array and make labels categorical
without_gender_savee_data_array = np.asarray(without_gender_savee_data)
without_gender_savee_label_array = np.array(without_gender_savee_labels)
# to_categorical(without_gender_savee_label_array)[0].shape
without_gender_savee_data_array.shape
without_gender_savee_label_array.shape

In [ ]:
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))
root_dir = "/kaggle/input/ravdess-emotional-speech-audio/audio_speech_actors_01-24/" 
##### load radvess speech data #####
# root_dir = "../input/audio_speech_actors_01-24/"
# root_dir = "../input/audio_speech_actors_01-24/"
# actor_dir = os.listdir("../input/audio_speech_actors_01-24/")
without_gender_radvess_speech_labels = []
without_gender_ravdess_speech_data = []
for actor_dir in sorted(os.listdir(root_dir)):
    actor_name = os.path.join(root_dir, actor_dir)
    for file in os.listdir(actor_name):
        if int(file[7:8]) == 1:
            continue
        if int(file[7:8]) == 2:
            continue
        else:
            without_gender_radvess_speech_labels.append(int(file[7:8]) - 1)
        #         radvess_speech_labels.append(classification(file))
            wav_file_name = os.path.join(root_dir, actor_dir, file)
            without_gender_ravdess_speech_data.append(extract_mfcc(wav_file_name))

In [ ]:
#### convert data to array and make labels categorical
without_gender_ravdess_speech_data_array = np.asarray(without_gender_ravdess_speech_data)
without_gender_ravdess_speech_label_array = np.array(without_gender_radvess_speech_labels)
without_gender_ravdess_speech_data_array.shape
without_gender_ravdess_speech_label_array.shape

In [ ]:
# for dirname, _, filenames in os.walk('/kaggle/input/'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

### load RAVDESS song data
root_dir = "../input/ravdess-emotional-song-audio/audio_song_actors_01-24/"
without_gender_radvess_song_labels = []
without_gender_ravdess_song_data = []
for actor_dir in sorted(os.listdir(root_dir)):
    actor_name = os.path.join(root_dir, actor_dir)
    for file in os.listdir(actor_name):
        if int(file[7:8]) == 1:
            continue
        if int(file[7:8]) == 2:
            continue
        else:
            without_gender_radvess_song_labels.append(int(file[7:8]) - 1)
        #         radvess_song_labels.append(classification(file))
            wav_file_name = os.path.join(root_dir, actor_dir, file)
            without_gender_ravdess_song_data.append(extract_mfcc(wav_file_name))

In [ ]:
#### convert data to array and make labels categorical
without_gender_ravdess_song_data_array = np.asarray(without_gender_ravdess_song_data)
without_gender_ravdess_song_label_array = np.array(without_gender_radvess_song_labels)
without_gender_ravdess_song_label_array.shape
without_gender_ravdess_song_data_array.shape

In [ ]:
# for dirname, _, filenames in os.walk('/kaggle/input/ravdess-audio-from-video-song/ravdess_audio_files_from_video_song/'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))


root_dir = "/kaggle/input/ravdess-audio-from-video-song/ravdess_audio_files_from_video_song/" 

without_gender_audio_only_data_song = [] ###stores the mfcc data
without_gender_audio_only_labels_song = [] ###stores the labels
for subdirs, dirs, files in os.walk(root_dir):
    for file in files:
        if int(file[7:8]) == 1:
            continue
        if int(file[7:8]) == 2:
            continue
        else:
            y, sr = librosa.load(os.path.join(subdirs,file), res_type='kaiser_best', 
                                     duration=librosa.get_duration(filename=os.path.join(subdirs,file)), offset=0.5)
            mfccs = np.mean(librosa.feature.mfcc(y=y, sr=sr, n_mfcc=40).T,axis=0)
            without_gender_audio_only_data_song.append(mfccs)
        #         audio_only_labels_song.append(classification(file))
            without_gender_audio_only_labels_song.append(int(file[7:8]) - 1)

In [ ]:
#### convert data to array and make labels categorical
without_gender_audio_only_data_song_array = np.array(without_gender_audio_only_data_song)
without_gender_audio_only_labels_song_array = np.array(without_gender_audio_only_labels_song)
without_gender_audio_only_data_song_array.shape
without_gender_audio_only_labels_song_array.shape

In [ ]:
# #### combine data
without_gender_data = np.r_[without_gender_audio_only_data_array, without_gender_cremad_only_data_array, without_gender_audio_only_data_song_array, without_gender_ravdess_song_data_array, without_gender_savee_data_array, without_gender_ravdess_speech_data_array]  # ravdess_song_data_array, audio_only_data_song_array,
without_gender_labels = np.r_[without_gender_audio_only_labels_array, without_gender_cremad_only_labels_array, without_gender_audio_only_labels_song_array, without_gender_ravdess_song_label_array, without_gender_savee_label_array, without_gender_ravdess_speech_label_array]  # ravdess_song_label_array, audio_only_labels_song_array,
# data = ravdess_speech_data_array
# labels = ravdess_speech_label_array
without_gender_labels.shape
without_gender_data.shape

In [ ]:
### make categorical labels
without_gender_labels_categorical = to_categorical(without_gender_labels)
without_gender_data.shape
without_gender_labels_categorical.shape

In [ ]:
def without_gender_create_model_LSTM():
    ### LSTM model, referred to the model A in the report
    model = Sequential()
    model.add(LSTM(128, return_sequences=False, input_shape=(40, 1)))
    model.add(Dense(64))
#     model.add(Dropout(0.4))
    model.add(Activation('relu'))
    model.add(Dense(32))
    model.add(Dropout(0.1))   ### 0.4
    model.add(Activation('relu'))
    model.add(Dense(8))
    model.add(Activation('softmax'))
    print ("model A : ")
    model.summary()
    model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])
    return model

def create_model_CNN():
    ### CNN model, referred to the model B in the report
    model = Sequential()
    model.add(Conv1D(8, kernel_size = 3, input_shape=(40, 1)))
    model.add(Activation('relu'))
    model.add(Conv1D(16,kernel_size = 3))
    model.add(Activation('relu'))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Conv1D(32, kernel_size = 3))
    model.add(Activation('relu'))
    model.add(Conv1D(16, kernel_size = 3))
    model.add(Activation('relu'))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Flatten())
    model.add(Dense(8))
    model.add(Activation('softmax'))
    print ("model B : ")
    model.summary()
    model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])
    return model

def new_CNN():
    ### CNN model, referred to the model C in the report
    model = Sequential()
    model.add(Conv1D(8, 5,padding='same', input_shape=(40, 1)))
    model.add(Activation('relu'))
    model.add(Conv1D(16, 5,padding='same'))
    model.add(Activation('relu'))
    model.add(Dropout(0.1))
    model.add(MaxPooling1D(pool_size=(8)))
    model.add(Conv1D(32, 5,padding='same',))
    model.add(Activation('relu'))
    model.add(Dropout(0.2))
    model.add(Conv1D(16, 5,padding='same',))
    model.add(Activation('relu'))
    model.add(Flatten())
    model.add(Dense(8))
    model.add(Activation('softmax'))
    opt = rmsprop(lr=0.00001, decay=1e-6)
    print ("model C : ")
    model.summary()
    model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
    return model

def train_CNN():
    ### CNN model, referred to the model D in the report
    model = Sequential()
    model.add(Conv1D(128, 5,padding='same', input_shape=(40,1)))
    model.add(Activation('relu'))
    model.add(Dropout(0.1))
    model.add(MaxPooling1D(pool_size=(8)))
    model.add(Conv1D(128, 5,padding='same',))
    model.add(Activation('relu'))
#     model.add(Dropout(0.1))
    model.add(Flatten())
    model.add(Dense(8))
    model.add(Activation('softmax'))
    opt = keras.optimizers.rmsprop(lr=0.00005, rho=0.9, epsilon=None, decay=0.0)
    print ("model D : ")
    model.summary()
    model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
    return model

def male_female_pitch():
    model = Sequential()
    model.add(Conv1D(256, 5, padding='same', input_shape=(40, 1)))
    model.add(Activation('relu'))
    model.add(Conv1D(128, 5, padding='same', ))
    model.add(Activation('relu'))
    model.add(Dropout(0.1))
    model.add(MaxPooling1D(pool_size=16))
    model.add(Conv1D(128, 5, padding='same', ))
    model.add(Activation('relu'))
    model.add(Conv1D(128, 5, padding='same', ))
    model.add(Activation('relu'))
    model.add(Conv1D(128, 5, padding='same', ))
    model.add(Activation('relu'))
    model.add(Dropout(0.1))
    model.add(Conv1D(128, 5, padding='same', ))
    model.add(Activation('relu'))
    model.add(Flatten())
    model.add(Dense(8))
    model.add(Activation('softmax'))
    opt = keras.optimizers.rmsprop(lr=0.00001, decay=1e-6)
    print ("model E : ")
    model.summary()
    model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])
    return model

def emotion_acc():
    model = Sequential()
    model.add(Conv1D(256, 5, padding='same', input_shape=(40, 1)))
    model.add(Activation('relu'))
    model.add(Conv1D(128, 5, padding='same', ))
    model.add(Activation('relu'))
    model.add(Dropout(0.1))
    model.add(MaxPooling1D(pool_size=16))
    model.add(Conv1D(128, 5, padding='same', ))
    model.add(Activation('relu'))
    model.add(Conv1D(128, 5, padding='same', ))
    model.add(Activation('relu'))
    model.add(Conv1D(128, 5, padding='same', ))
    model.add(Activation('relu'))
    model.add(Dropout(0.1))
    model.add(Conv1D(128, 5, padding='same', ))
    model.add(Activation('relu'))
    model.add(Flatten())
    model.add(Dense(8))
    model.add(Activation('softmax'))
    opt = keras.optimizers.rmsprop(lr=0.00001, decay=1e-6)
    print ("model F : ")
    model.summary()
    model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
    return model

In [ ]:
# np.random.shuffle(data)
without_gender_number_of_samples = without_gender_data.shape[0]
without_gender_training_samples = int(without_gender_number_of_samples * 0.8)
without_gender_validation_samples = int(without_gender_training_samples * 0.33)
without_gender_test_samples = int(without_gender_number_of_samples-(without_gender_training_samples))
without_gender_class_weights = class_weight.compute_class_weight('balanced', np.unique(without_gender_labels_categorical[without_gender_training_samples]),
                                                  without_gender_labels_categorical[without_gender_training_samples])
print (without_gender_training_samples)
print (without_gender_validation_samples)
print (without_gender_test_samples)

In [ ]:
### train using model A
model_without_gender = male_female_pitch()  # without_gender_create_model_LSTM()
history = model_without_gender.fit(np.expand_dims(without_gender_data[:without_gender_training_samples],-1), 
                      without_gender_labels_categorical[:without_gender_training_samples], 
                      validation_data=(np.expand_dims(without_gender_data[without_gender_validation_samples:], -1), 
                      without_gender_labels_categorical[without_gender_validation_samples:]),
                      class_weight=without_gender_class_weights, epochs=300, shuffle=False)  # class_weight=without_gender_class_weights, 

In [ ]:
### loss plots using model A
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(1, len(loss) + 1)

plt.plot(epochs, loss, 'ro', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.show()

In [ ]:
### accuracy plots using model A
plt.clf()                                                

acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

plt.plot(epochs, acc, 'ro', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.show()

In [ ]:
j = 1
print ("CNN {0:d}: Epochs={1:d}, Train accuracy={2:.5f}, Validation accuracy={3:.5f}".format(
        j + 1, 100, max(history.history['accuracy']), max(history.history['val_accuracy'])))
# predictions = model_A.predict(np.expand_dims(data[test_samples:], -1))
# predictions[0:5]
train_acc = model_without_gender.evaluate(np.expand_dims(without_gender_data[:without_gender_training_samples], -1), without_gender_labels_categorical[:without_gender_training_samples], verbose=0)
test_acc = model_without_gender.evaluate(np.expand_dims(without_gender_data[without_gender_test_samples:], -1),
                                without_gender_labels_categorical[without_gender_test_samples:], verbose=0)
val_acc = model_without_gender.evaluate(np.expand_dims(without_gender_data[without_gender_validation_samples:], -1), without_gender_labels_categorical[without_gender_validation_samples:], verbose=0)
print ("accuracy printing")
print (train_acc)
print (test_acc)
print (val_acc)

In [ ]:
###############With gender detection starts#######################

In [ ]:
# ### although, we load the data here, it is not used in training or validation
# for dirname, _, filenames in os.walk('/kaggle/input/audiowav-cremad/AudioWAV_cremad/'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))
root_dir = "/kaggle/input/audiowav-cremad/AudioWAV_cremad/" 

cremad_only_data = [] ###stores the mfcc data
cremad_only_labels = [] ###stores the labels
for subdirs, dirs, files in os.walk(root_dir):
    for file in files:
        if file[11:14] == 'NEU':
            continue
        else:
            y, sr = librosa.load(os.path.join(subdirs,file), res_type='kaiser_best', 
                                 duration=librosa.get_duration(filename=os.path.join(subdirs,file)), offset=0.25)
            mfccs = np.mean(librosa.feature.mfcc(y=y, sr=sr, n_mfcc=40).T,axis=0)
            cremad_only_data.append(mfccs)
            cremad_only_labels.append(genderClassification_another(file))

In [ ]:
#### convert data to array and make labels categorical
cremad_only_data_array = np.array(cremad_only_data)
cremad_only_labels_array = np.array(cremad_only_labels)
cremad_only_data_array.shape

In [ ]:
##### load data from savee dataset
#### although, we load the data here, it is not used in training or validation
# for dirname, _, filenames in os.walk('/kaggle/input/savee-database/audiodata/AudioData/'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))
root_dir = "/kaggle/input/savee-database/audiodata/AudioData/"
# root_dir = "../input/audio_speech_actors_01-24/"
savee_data = []
savee_labels = []
for actor_dir in sorted(os.listdir(root_dir)):
    if actor_dir[-4:] == ".txt":
        continue
    for file_name in os.listdir(os.path.join(root_dir, actor_dir)):
        if file_name[0] == "c":
            continue
        if file_name[0] == "n":
            continue
        else:
    #         gender_list = 1
            savee_labels.append(0)  # male
            wav_file_name = os.path.join(root_dir, actor_dir, file_name)
            savee_data.append(extract_mfcc(wav_file_name))

In [ ]:
#### convert data to array and make labels categorical
savee_data_array = np.asarray(savee_data)
savee_label_array = np.array(savee_labels)
to_categorical(savee_label_array)[0].shape
# savee_data_array.shape

In [ ]:
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))
root_dir = "/kaggle/input/ravdess-emotional-speech-audio/audio_speech_actors_01-24/" 
##### load radvess speech data #####
# root_dir = "../input/audio_speech_actors_01-24/"
# root_dir = "../input/audio_speech_actors_01-24/"
# actor_dir = os.listdir("../input/audio_speech_actors_01-24/")
radvess_speech_labels = []
ravdess_speech_data = []
for actor_dir in sorted(os.listdir(root_dir)):
    actor_name = os.path.join(root_dir, actor_dir)
    for file in os.listdir(actor_name):
        if int(file[7:8]) == 1:
            continue
        if int(file[7:8]) == 2:
            continue
        else:
            radvess_speech_labels.append(genderClassification(file))
            wav_file_name = os.path.join(root_dir, actor_dir, file)
            ravdess_speech_data.append(extract_mfcc(wav_file_name))

In [ ]:
#### convert data to array and make labels categorical
ravdess_speech_data_array = np.asarray(ravdess_speech_data)
ravdess_speech_label_array = np.array(radvess_speech_labels)
ravdess_speech_label_array.shape

In [ ]:
# for dirname, _, filenames in os.walk('/kaggle/input/ravdessaudiofilessfromvideo/ravdess_audio_files_from_video/'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

### extract audio data from AV RAVDESS data
root_dir = "/kaggle/input/ravdessaudiofilessfromvideo/ravdess_audio_files_from_video/" 

audio_only_data = [] ###stores the mfcc data
audio_only_labels = [] ###stores the labels
for subdirs, dirs, files in os.walk(root_dir):
    for file in files:
        if int(file[7:8]) == 1:
            continue
        if int(file[7:8]) == 2:
            continue
        else:
            y, sr = librosa.load(os.path.join(subdirs,file), res_type='kaiser_best', 
                                 duration=librosa.get_duration(filename=os.path.join(subdirs,file)), offset=0.5)
            mfccs = np.mean(librosa.feature.mfcc(y=y, sr=sr, n_mfcc=40).T,axis=0)
            audio_only_data.append(mfccs)
            audio_only_labels.append(genderClassification(file))
    #         audio_only_labels.append(int(file[7:8]) - 1)

In [ ]:
#### convert data to array and make labels categorical
audio_only_data_array = np.array(audio_only_data)
audio_only_labels_array = np.array(audio_only_labels)
audio_only_data_array.shape

In [ ]:
# for dirname, _, filenames in os.walk('/kaggle/input/ravdess-audio-from-video-song/ravdess_audio_files_from_video_song/'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

### extract audio data from AV RAVDESS data
root_dir = "/kaggle/input/ravdess-audio-from-video-song/ravdess_audio_files_from_video_song/" 

audio_only_data_song = [] ###stores the mfcc data
audio_only_labels_song = [] ###stores the labels
for subdirs, dirs, files in os.walk(root_dir):
    for file in files:
        if int(file[7:8]) == 1:
            continue
        if int(file[7:8]) == 2:
            continue
        else:
            y, sr = librosa.load(os.path.join(subdirs,file), res_type='kaiser_best', 
                                 duration=librosa.get_duration(filename=os.path.join(subdirs,file)), offset=0.5)
            mfccs = np.mean(librosa.feature.mfcc(y=y, sr=sr, n_mfcc=40).T,axis=0)
            audio_only_data_song.append(mfccs)
            audio_only_labels_song.append(genderClassification(file))
    #         audio_only_labels_song.append(int(file[7:8]) - 1)

In [ ]:
#### convert data to array and make labels categorical
audio_only_data_song_array = np.array(audio_only_data_song)
audio_only_labels_song_array = np.array(audio_only_labels_song)
audio_only_data_song_array.shape

In [ ]:
# for dirname, _, filenames in os.walk('/kaggle/input/'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

### load RAVDESS song data
root_dir = "../input/ravdess-emotional-song-audio/audio_song_actors_01-24/"
radvess_song_labels = []
ravdess_song_data = []
for actor_dir in sorted(os.listdir(root_dir)):
    actor_name = os.path.join(root_dir, actor_dir)
    for file in os.listdir(actor_name):
        if int(file[7:8]) == 1:
            continue
        if int(file[7:8]) == 2:
            continue
        else:
    #         radvess_song_labels.append(int(file[7:8]) - 1)
            radvess_song_labels.append(genderClassification(file))
            wav_file_name = os.path.join(root_dir, actor_dir, file)
            ravdess_song_data.append(extract_mfcc(wav_file_name))

In [ ]:
#### convert data to array and make labels categorical
ravdess_song_data_array = np.asarray(ravdess_song_data)
ravdess_song_label_array = np.array(radvess_song_labels)
ravdess_song_label_array.shape

In [ ]:
# #### combine data
data = np.r_[audio_only_data_array, cremad_only_data_array, ravdess_song_data_array, savee_data_array, audio_only_data_song_array, ravdess_speech_data_array]  # ravdess_song_data_array, audio_only_data_song_array,
labels = np.r_[audio_only_labels_array, cremad_only_labels_array, ravdess_song_label_array, savee_label_array, audio_only_labels_song_array, ravdess_speech_label_array]  # ravdess_song_label_array, audio_only_labels_song_array,
# data = ravdess_speech_data_array
# labels = ravdess_speech_label_array
labels.shape

In [ ]:
### make categorical labels
labels_categorical = to_categorical(labels)
data.shape
labels_categorical.shape

In [9]:
def create_model_LSTM():
    ### LSTM model, referred to the model A in the report
    model = Sequential()
    model.add(LSTM(128, return_sequences=False, input_shape=(40, 1)))
    model.add(Dense(64))
    model.add(Dropout(0.4))
    model.add(Activation('relu'))
    model.add(Dense(32))
    model.add(Dropout(0.4))   ### 0.4
    model.add(Activation('relu'))
    model.add(Dense(2))
    model.add(Activation('softmax'))
    print ("model A : ")
    model.summary()
    model.compile(loss='binary_crossentropy', optimizer='Adam', metrics=['accuracy'])
    return model

def male_female_pitch():
    model = Sequential()
    model.add(Conv1D(256, 5, padding='same', input_shape=(40, 1)))
    model.add(Activation('relu'))
    model.add(Conv1D(128, 5, padding='same', ))
    model.add(Activation('relu'))
    model.add(Dropout(0.1))
    model.add(MaxPooling1D(pool_size=16))
    model.add(Conv1D(128, 5, padding='same', ))
    model.add(Activation('relu'))
    model.add(Conv1D(128, 5, padding='same', ))
    model.add(Activation('relu'))
    model.add(Conv1D(128, 5, padding='same', ))
    model.add(Activation('relu'))
    model.add(Dropout(0.1))
    model.add(Conv1D(128, 5, padding='same', ))
    model.add(Activation('relu'))
    model.add(Flatten())
    model.add(Dense(2))
    model.add(Activation('softmax'))
    opt = keras.optimizers.rmsprop(lr=0.00001, decay=1e-6)
    print ("model B : ")
    model.summary()
    model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])
    return model

In [ ]:
# np.random.shuffle(data)
number_of_samples = data.shape[0]
training_samples = int(number_of_samples * 0.8)
validation_samples = int(training_samples * 0.33)
test_samples = int(number_of_samples-(training_samples))
print (training_samples)
print (validation_samples)
print (test_samples)
# class_weights = class_weight.compute_class_weight('balanced', np.unique(labels_categorical[training_samples]),
#                                                   labels_categorical[training_samples])
# male_female_pitch()

In [10]:
model_B = create_model_LSTM()  # male_female_pitch()  # create_model_LSTM()
history = model_B.fit(np.expand_dims(data[:training_samples],-1), 
                      labels_categorical[:training_samples], 
                      validation_data=(np.expand_dims(data[validation_samples:], -1), 
                                       labels_categorical[validation_samples:]), 
                      epochs=300, shuffle=False)  # class_weight=class_weights, 

model A : 
Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_2 (LSTM)                (None, 128)               66560     
_________________________________________________________________
dense_4 (Dense)              (None, 64)                8256      
_________________________________________________________________
dropout_2 (Dropout)          (None, 64)                0         
_________________________________________________________________
activation_4 (Activation)    (None, 64)                0         
_________________________________________________________________
dense_5 (Dense)              (None, 32)                2080      
_________________________________________________________________
dropout_3 (Dropout)          (None, 32)                0         
_________________________________________________________________
activation_5 (Activation)    (None, 32)    

In [ ]:
### loss plots using model A
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(1, len(loss) + 1)

plt.plot(epochs, loss, 'ro', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.show()

In [ ]:
### accuracy plots using model A
plt.clf()                                                

acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

plt.plot(epochs, acc, 'ro', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

plt.show()

In [ ]:
j = 1
print ("CNN {0:d}: Epochs={1:d}, Train accuracy={2:.5f}, Validation accuracy={3:.5f}".format(
        j + 1, 100, max(history.history['accuracy']), max(history.history['val_accuracy'])))
print ("CNN {0:d}: Epochs={1:d}, Train loss={2:.5f}, Validation loss={3:.5f}".format(
        j + 1, 100, min(history.history['loss']), min(history.history['val_loss'])))
# predictions = model_A.predict(np.expand_dims(data[test_samples:], -1))
# predictions[0:5]
train_acc = model_B.evaluate(np.expand_dims(data[:training_samples], -1), labels_categorical[:training_samples], verbose=0)
test_acc = model_B.evaluate(np.expand_dims(data[test_samples:], -1),
                                labels_categorical[test_samples:], verbose=0)
val_acc = model_B.evaluate(np.expand_dims(data[validation_samples:], -1), labels_categorical[validation_samples:], verbose=0)
print ("accuracy printing")
print (train_acc)
print (test_acc)
print (val_acc)

In [ ]:
################female gender emotion#######################

In [ ]:
# ### although, we load the data here, it is not used in training or validation
# for dirname, _, filenames in os.walk('/kaggle/input/audiowav-cremad/AudioWAV_cremad/'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))
root_dir = "/kaggle/input/audiowav-cremad/AudioWAV_cremad/" 

female_cremad_only_data = [] ###stores the mfcc data
female_cremad_only_labels = [] ###stores the labels
for subdirs, dirs, files in os.walk(root_dir):
    for file in files:
        if file[0] == 'f':
            if file[11:14] == 'NEU':
                continue
            else:
                y, sr = librosa.load(os.path.join(subdirs,file), res_type='kaiser_best', 
                                         duration=librosa.get_duration(filename=os.path.join(subdirs,file)),
                                         offset=0.25)
                mfccs = np.mean(librosa.feature.mfcc(y=y, sr=sr, n_mfcc=40).T,axis=0)
                female_cremad_only_data.append(mfccs)
                female_cremad_only_labels.append(classification(file))

In [ ]:
#### convert data to array and make labels categorical
female_cremad_only_data_array = np.array(female_cremad_only_data)
female_cremad_only_labels_array = np.array(female_cremad_only_labels)
female_cremad_only_data_array.shape

In [ ]:
# for dirname, _, filenames in os.walk('/kaggle/input/ravdessaudiofilessfromvideo/ravdess_audio_files_from_video/'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

### extract audio data from AV RAVDESS data
root_dir = "/kaggle/input/ravdessaudiofilessfromvideo/ravdess_audio_files_from_video/" 

female_audio_only_data = []  # stores the mfcc data
female_audio_only_labels = []  # stores the labels
for subdirs, dirs, files in os.walk(root_dir):
    for file in files:
        if int(file[18:-4]) % 2 == 0:
            if int(file[7:8]) == 1:
                continue
            if int(file[7:8]) == 2:
                continue
            else:
                y, sr = librosa.load(os.path.join(subdirs, file), res_type='kaiser_best', 
                                     duration=librosa.get_duration(filename=os.path.join(subdirs,file)),
                                     offset=0.5)
                mfccs = np.mean(librosa.feature.mfcc(y=y, sr=sr, n_mfcc=40).T, axis=0)
                female_audio_only_data.append(mfccs)
                female_audio_only_labels.append(int(file[7:8]) - 1)
                #         female_audio_only_labels.append(classification(file))

In [ ]:
#### convert data to array and make labels categorical
female_audio_only_data_array = np.array(female_audio_only_data)
female_audio_only_labels_array = np.array(female_audio_only_labels)
female_audio_only_data_array.shape

In [ ]:
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))
root_dir = "/kaggle/input/ravdess-emotional-speech-audio/audio_speech_actors_01-24/" 
##### load radvess speech data #####
# root_dir = "../input/audio_speech_actors_01-24/"
# root_dir = "../input/audio_speech_actors_01-24/"
# actor_dir = os.listdir("../input/audio_speech_actors_01-24/")
female_radvess_speech_labels = []
female_ravdess_speech_data = []
for actor_dir in sorted(os.listdir(root_dir)):
    actor_name = os.path.join(root_dir, actor_dir)
    for file in os.listdir(actor_name):
        if int(file[18:-4]) % 2 == 0:
            if int(file[7:8]) == 1:
                continue
            if int(file[7:8]) == 2:
                continue
            else:
                female_radvess_speech_labels.append(int(file[7:8]) - 1)
        #         female_radvess_speech_labels.append(classification(file))
                wav_file_name = os.path.join(root_dir, actor_dir, file)
                female_ravdess_speech_data.append(extract_mfcc(wav_file_name))

In [ ]:
#### convert data to array and make labels categorical
female_ravdess_speech_data_array = np.asarray(female_ravdess_speech_data)
female_ravdess_speech_label_array = np.array(female_radvess_speech_labels)
female_ravdess_speech_label_array.shape

In [ ]:
# for dirname, _, filenames in os.walk('/kaggle/input/'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

### load RAVDESS song data
root_dir = "../input/ravdess-emotional-song-audio/audio_song_actors_01-24/"
female_radvess_song_labels = []
female_ravdess_song_data = []
for actor_dir in sorted(os.listdir(root_dir)):
    actor_name = os.path.join(root_dir, actor_dir)
    for file in os.listdir(actor_name):
        if int(file[18:-4]) % 2 == 0:
            if int(file[7:8]) == 1:
                continue
            if int(file[7:8]) == 2:
                continue
            else:
                female_radvess_song_labels.append(int(file[7:8]) - 1)
        #         female_radvess_song_labels.append(classification(file))
                wav_file_name = os.path.join(root_dir, actor_dir, file)
                female_ravdess_song_data.append(extract_mfcc(wav_file_name))

In [ ]:
#### convert data to array and make labels categorical
female_ravdess_song_data_array = np.asarray(female_ravdess_song_data)
female_ravdess_song_label_array = np.array(female_radvess_song_labels)
female_ravdess_song_label_array.shape

In [ ]:
# for dirname, _, filenames in os.walk('/kaggle/input/ravdess-audio-from-video-song/ravdess_audio_files_from_video_song/'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

root_dir = "/kaggle/input/ravdess-audio-from-video-song/ravdess_audio_files_from_video_song/" 

female_audio_only_data_song = [] ###stores the mfcc data
female_audio_only_labels_song = [] ###stores the labels
for subdirs, dirs, files in os.walk(root_dir):
    for file in files:
        if int(file[18:-4]) % 2 == 0:
            if int(file[7:8]) == 1:
                continue
            if int(file[7:8]) == 2:
                continue
            else:
                y, sr = librosa.load(os.path.join(subdirs,file), res_type='kaiser_best', 
                                         duration=librosa.get_duration(filename=os.path.join(subdirs,file)), 
                                     offset=0.5)
                mfccs = np.mean(librosa.feature.mfcc(y=y, sr=sr, n_mfcc=40).T,axis=0)
                female_audio_only_data_song.append(mfccs)
            #         female_audio_only_labels_song.append(classification(file))
                female_audio_only_labels_song.append(int(file[7:8]) - 1)

In [ ]:
#### convert data to array and make labels categorical
female_audio_only_data_song_array = np.array(female_audio_only_data_song)
female_audio_only_labels_song_array = np.array(female_audio_only_labels_song)
female_audio_only_data_song_array.shape

In [ ]:
# #### combine data
female_data = np.r_[female_audio_only_data_array, female_cremad_only_data_array, female_ravdess_song_data_array, female_audio_only_data_song_array, female_ravdess_speech_data_array]  # savee_data_array,--> only male # ravdess_song_data_array, audio_only_data_song_array,
female_labels = np.r_[female_audio_only_labels_array, female_cremad_only_labels_array, female_ravdess_song_label_array, female_audio_only_labels_song_array, female_ravdess_speech_label_array]  # savee_label_array,--> only male #ravdess_song_label_array, audio_only_labels_song_array,
# data = ravdess_speech_data_array
# labels = ravdess_speech_label_array
female_data.shape
female_labels.shape

In [ ]:
### make categorical labels
female_labels_categorical = to_categorical(female_labels)
female_data.shape
female_labels_categorical.shape

In [11]:
def female_create_model_LSTM():
    ### LSTM model, referred to the model A in the report
    model = Sequential()
    model.add(LSTM(128, return_sequences=False, input_shape=(40, 1)))
    model.add(Dense(64))
    model.add(Dropout(0.4))
    model.add(Activation('relu'))
    model.add(Dense(32))
    model.add(Dropout(0.4))   ### 0.4
    model.add(Activation('relu'))
    model.add(Dense(8))
    model.add(Activation('softmax'))
    print ("model A : ")
    model.summary()
    model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])
    return model

def male_female_pitch():
    model = Sequential()
    model.add(Conv1D(256, 5, padding='same', input_shape=(40, 1)))
    model.add(Activation('relu'))
    model.add(Conv1D(128, 5, padding='same', ))
    model.add(Activation('relu'))
    model.add(Dropout(0.1))
    model.add(MaxPooling1D(pool_size=16))
    model.add(Conv1D(128, 5, padding='same', ))
    model.add(Activation('relu'))
    model.add(Conv1D(128, 5, padding='same', ))
    model.add(Activation('relu'))
    model.add(Conv1D(128, 5, padding='same', ))
    model.add(Activation('relu'))
    model.add(Dropout(0.1))
    model.add(Conv1D(128, 5, padding='same', ))
    model.add(Activation('relu'))
    model.add(Flatten())
    model.add(Dense(8))
    model.add(Activation('softmax'))
    opt = keras.optimizers.rmsprop(lr=0.00001, decay=1e-6)
    print ("model B : ")
    model.summary()
    model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])
    return model

In [13]:
# np.random.shuffle(data)
female_number_of_samples = female_data.shape[0]
female_training_samples = int(female_number_of_samples * 0.8)
female_validation_samples = int(female_training_samples * 0.33)
female_test_samples = int(female_number_of_samples-(female_training_samples))
female_class_weights = class_weight.compute_class_weight('balanced', np.unique(female_labels_categorical[female_training_samples]),
                                                  female_labels_categorical[female_training_samples])
print (female_training_samples)
print (female_validation_samples)
print (female_test_samples)
# print (female_class_weights)
# male_female_pitch()

model B : 
Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_1 (Conv1D)            (None, 40, 256)           1536      
_________________________________________________________________
activation_10 (Activation)   (None, 40, 256)           0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 40, 128)           163968    
_________________________________________________________________
activation_11 (Activation)   (None, 40, 128)           0         
_________________________________________________________________
dropout_6 (Dropout)          (None, 40, 128)           0         
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 2, 128)            0         
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 2, 128)

In [12]:
### train using model A
model_A = female_create_model_LSTM()  # male_female_pitch()
history = model_A.fit(np.expand_dims(female_data[:female_training_samples],-1), 
                      female_labels_categorical[:female_training_samples], 
                      validation_data=(np.expand_dims(female_data[female_validation_samples:], -1), 
                                       female_labels_categorical[female_validation_samples:]), 
                      class_weight=female_class_weights, epochs=250, shuffle=True)  # class_weight=female_class_weights, 

model A : 
Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_3 (LSTM)                (None, 128)               66560     
_________________________________________________________________
dense_7 (Dense)              (None, 64)                8256      
_________________________________________________________________
dropout_4 (Dropout)          (None, 64)                0         
_________________________________________________________________
activation_7 (Activation)    (None, 64)                0         
_________________________________________________________________
dense_8 (Dense)              (None, 32)                2080      
_________________________________________________________________
dropout_5 (Dropout)          (None, 32)                0         
_________________________________________________________________
activation_8 (Activation)    (None, 32)    

In [ ]:
### loss plots using model A
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(1, len(loss) + 1)

plt.plot(epochs, loss, 'ro', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.show()

In [ ]:
### accuracy plots using model A
plt.clf()                                                

acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

plt.plot(epochs, acc, 'ro', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.show()

In [ ]:
j = 1
print ("CNN {0:d}: Epochs={1:d}, Train accuracy={2:.5f}, Validation accuracy={3:.5f}".format(
        j + 1, 100, max(history.history['accuracy']), max(history.history['val_accuracy'])))
# predictions = model_A.predict(np.expand_dims(data[test_samples:], -1))
# predictions[0:5]
train_acc = model_A.evaluate(np.expand_dims(female_data[:female_training_samples], -1), female_labels_categorical[:female_training_samples], verbose=0)
test_acc = model_A.evaluate(np.expand_dims(female_data[female_test_samples:], -1),
                                female_labels_categorical[female_test_samples:], verbose=0)
val_acc = model_A.evaluate(np.expand_dims(female_data[female_validation_samples:], -1), female_labels_categorical[female_validation_samples:], verbose=0)
print ("accuracy printing")
print (train_acc)
print (test_acc)
print (val_acc)

In [ ]:
####################male gender detection#################

In [ ]:
# ### although, we load the data here, it is not used in training or validation
# for dirname, _, filenames in os.walk('/kaggle/input/audiowav-cremad/AudioWAV_cremad/'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))
root_dir = "/kaggle/input/audiowav-cremad/AudioWAV_cremad/" 

male_cremad_only_data = [] ###stores the mfcc data
male_cremad_only_labels = [] ###stores the labels
for subdirs, dirs, files in os.walk(root_dir):
    for file in files:
        if file[0] == 'm':
            if file[11:14] == 'NEU':
                    continue
            else:
                y, sr = librosa.load(os.path.join(subdirs,file), res_type='kaiser_best', 
                                         duration=librosa.get_duration(filename=os.path.join(subdirs,file)), 
                                     offset=0.25)
                mfccs = np.mean(librosa.feature.mfcc(y=y, sr=sr, n_mfcc=40).T,axis=0)
                male_cremad_only_data.append(mfccs)
                male_cremad_only_labels.append(classification(file))

In [ ]:
#### convert data to array and make labels categorical
male_cremad_only_data_array = np.array(male_cremad_only_data)
male_cremad_only_labels_array = np.array(male_cremad_only_labels)
male_cremad_only_data_array.shape

In [ ]:
# for dirname, _, filenames in os.walk('/kaggle/input/ravdessaudiofilessfromvideo/ravdess_audio_files_from_video/'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

### extract audio data from AV RAVDESS data
root_dir = "/kaggle/input/ravdessaudiofilessfromvideo/ravdess_audio_files_from_video/" 

male_audio_only_data = []  # stores the mfcc data
male_audio_only_labels = []  # stores the labels
for subdirs, dirs, files in os.walk(root_dir):
    for file in files:
        if int(file[18:-4]) % 2 == 1:
            if int(file[7:8]) == 1:
                    continue
            if int(file[7:8]) == 2:
                continue
            else:
                y, sr = librosa.load(os.path.join(subdirs, file), res_type='kaiser_best', 
                                 duration=librosa.get_duration(filename=os.path.join(subdirs,file)), 
                                offset=0.5)
                mfccs = np.mean(librosa.feature.mfcc(y=y, sr=sr, n_mfcc=40).T, axis=0)
                male_audio_only_data.append(mfccs)
                male_audio_only_labels.append(int(file[7:8]) - 1)
                 #         male_audio_only_labels.append(classification(file))

In [ ]:
#### convert data to array and make labels categorical
male_audio_only_data_array = np.array(male_audio_only_data)
male_audio_only_labels_array = np.array(male_audio_only_labels)
male_audio_only_data_array.shape
male_audio_only_labels_array.shape

In [ ]:
##### load data from savee dataset
#### although, we load the data here, it is not used in training or validation
# for dirname, _, filenames in os.walk('/kaggle/input/savee-database/audiodata/AudioData/'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))
root_dir = "/kaggle/input/savee-database/audiodata/AudioData/"
# root_dir = "../input/audio_speech_actors_01-24/"
male_savee_data = []
male_savee_labels = []
for actor_dir in sorted(os.listdir(root_dir)):
    if actor_dir[-4:] == ".txt":
        continue
    for file_name in os.listdir(os.path.join(root_dir, actor_dir)):
        if file_name[0] == "c":
            continue
        if file_name[0] == "n":
#             male_savee_labels.append(0)
            continue
        wav_file_name = os.path.join(root_dir, actor_dir, file_name)
        male_savee_data.append(extract_mfcc(wav_file_name))
#         male_savee_labels.append(classification(file_name))
        if file_name[0] == "a":
            male_savee_labels.append(4)
        if file_name[0] == "d":
            male_savee_labels.append(6)
        if file_name[0] == "f":
            male_savee_labels.append(5)
        if file_name[0] == "h":
            male_savee_labels.append(2)
        if file_name[:2] == "sa":
            male_savee_labels.append(3)
        if file_name[:2] == "su":
            male_savee_labels.append(7)

In [ ]:
#### convert data to array and make labels categorical
male_savee_data_array = np.asarray(male_savee_data)
male_savee_label_array = np.array(male_savee_labels)
# to_categorical(male_savee_label_array)[0].shape
male_savee_data_array.shape
male_savee_label_array.shape

In [ ]:
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))
root_dir = "/kaggle/input/ravdess-emotional-speech-audio/audio_speech_actors_01-24/" 
##### load radvess speech data #####
# root_dir = "../input/audio_speech_actors_01-24/"
# root_dir = "../input/audio_speech_actors_01-24/"
# actor_dir = os.listdir("../input/audio_speech_actors_01-24/")
male_radvess_speech_labels = []
male_ravdess_speech_data = []
for actor_dir in sorted(os.listdir(root_dir)):
    actor_name = os.path.join(root_dir, actor_dir)
    for file in os.listdir(actor_name):
        if int(file[18:-4]) % 2 == 1:
            if int(file[7:8]) == 1:
                continue
            if int(file[7:8]) == 2:
                continue
            else:
                male_radvess_speech_labels.append(int(file[7:8]) - 1)
        #         male_radvess_speech_labels.append(classification(file))
                wav_file_name = os.path.join(root_dir, actor_dir, file)
                male_ravdess_speech_data.append(extract_mfcc(wav_file_name))

In [ ]:
#### convert data to array and make labels categorical
male_ravdess_speech_data_array = np.asarray(male_ravdess_speech_data)
male_ravdess_speech_label_array = np.array(male_radvess_speech_labels)
male_ravdess_speech_data_array.shape
male_ravdess_speech_label_array.shape

In [ ]:
# for dirname, _, filenames in os.walk('/kaggle/input/'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

### load RAVDESS song data
root_dir = "../input/ravdess-emotional-song-audio/audio_song_actors_01-24/"
male_radvess_song_labels = []
male_ravdess_song_data = []
for actor_dir in sorted(os.listdir(root_dir)):
    actor_name = os.path.join(root_dir, actor_dir)
    for file in os.listdir(actor_name):
        if int(file[18:-4]) % 2 == 1:
            if int(file[7:8]) == 1:
                continue
            if int(file[7:8]) == 2:
                continue
            else:
                male_radvess_song_labels.append(int(file[7:8]) - 1)
        #         male_radvess_song_labels.append(classification(file))
                wav_file_name = os.path.join(root_dir, actor_dir, file)
                male_ravdess_song_data.append(extract_mfcc(wav_file_name))

In [ ]:
#### convert data to array and make labels categorical
male_ravdess_song_data_array = np.asarray(male_ravdess_song_data)
male_ravdess_song_label_array = np.array(male_radvess_song_labels)
male_ravdess_song_label_array.shape
male_ravdess_song_data_array.shape

In [ ]:
# for dirname, _, filenames in os.walk('/kaggle/input/ravdess-audio-from-video-song/ravdess_audio_files_from_video_song/'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))


root_dir = "/kaggle/input/ravdess-audio-from-video-song/ravdess_audio_files_from_video_song/" 

male_audio_only_data_song = [] ###stores the mfcc data
male_audio_only_labels_song = [] ###stores the labels
for subdirs, dirs, files in os.walk(root_dir):
    for file in files:
        if int(file[18:-4]) % 2 == 1:
            if int(file[7:8]) == 1:
                continue
            if int(file[7:8]) == 2:
                continue
            else:
                y, sr = librosa.load(os.path.join(subdirs,file), res_type='kaiser_best', 
                                     duration=librosa.get_duration(filename=os.path.join(subdirs,file)), 
                                     offset=0.5)
                mfccs = np.mean(librosa.feature.mfcc(y=y, sr=sr, n_mfcc=40).T,axis=0)
                male_audio_only_data_song.append(mfccs)
        #         male_audio_only_labels_song.append(classification(file))
                male_audio_only_labels_song.append(int(file[7:8]) - 1)

In [ ]:
#### convert data to array and make labels categorical
male_audio_only_data_song_array = np.array(male_audio_only_data_song)
male_audio_only_labels_song_array = np.array(male_audio_only_labels_song)
male_audio_only_data_song_array.shape
male_audio_only_labels_song_array.shape

In [ ]:
# #### combine data
male_data = np.r_[male_audio_only_data_array, male_cremad_only_data_array, male_audio_only_data_song_array, male_ravdess_song_data_array, male_savee_data_array, male_ravdess_speech_data_array]  # ravdess_song_data_array, audio_only_data_song_array,
male_labels = np.r_[male_audio_only_labels_array, male_cremad_only_labels_array, male_audio_only_labels_song_array, male_ravdess_song_label_array, male_savee_label_array, male_ravdess_speech_label_array]  # ravdess_song_label_array, audio_only_labels_song_array,
# male_data = ravdess_speech_data_array
# male_labels = ravdess_speech_label_array
male_labels.shape
male_data.shape

In [ ]:
### make categorical labels
male_labels_categorical = to_categorical(male_labels)
male_data.shape
male_labels_categorical.shape

In [ ]:
def male_create_model_LSTM():
    ### LSTM model, referred to the model A in the report
    model = Sequential()
    model.add(LSTM(128, return_sequences=False, input_shape=(40, 1)))
    model.add(Dense(64))
#     model.add(Dropout(0.4))
    model.add(Activation('relu'))
    model.add(Dense(32))
    model.add(Dropout(0.1))   ### 0.4
    model.add(Activation('relu'))
    model.add(Dense(8))
    model.add(Activation('softmax'))
    print ("model C : ")
    model.summary()
    model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])
    return model

def male_female_pitch():
    model = Sequential()
    model.add(Conv1D(256, 5, padding='same', input_shape=(40, 1)))
    model.add(Activation('relu'))
    model.add(Conv1D(128, 5, padding='same', ))
    model.add(Activation('relu'))
    model.add(Dropout(0.1))
    model.add(MaxPooling1D(pool_size=16))
    model.add(Conv1D(128, 5, padding='same', ))
    model.add(Activation('relu'))
    model.add(Conv1D(128, 5, padding='same', ))
    model.add(Activation('relu'))
    model.add(Conv1D(128, 5, padding='same', ))
    model.add(Activation('relu'))
    model.add(Dropout(0.1))
    model.add(Conv1D(128, 5, padding='same', ))
    model.add(Activation('relu'))
    model.add(Flatten())
    model.add(Dense(8))
    model.add(Activation('softmax'))
    opt = keras.optimizers.rmsprop(lr=0.00001, decay=1e-6)
    print ("model E : ")
    model.summary()
    model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])
    return model

In [ ]:
# np.random.shuffle(data)
male_number_of_samples = male_data.shape[0]
male_training_samples = int(male_number_of_samples * 0.8)
male_validation_samples = int(male_training_samples * 0.33)
male_test_samples = int(male_number_of_samples-(male_training_samples))
male_class_weights = class_weight.compute_class_weight('balanced', np.unique(male_labels_categorical[male_training_samples]),
                                                  male_labels_categorical[male_training_samples])
print (male_training_samples)
print (male_validation_samples)
print (male_test_samples)

In [ ]:
### train using model A
model_C = male_female_pitch()  # male_create_model_LSTM()
history = model_C.fit(np.expand_dims(male_data[:male_training_samples],-1), 
                      male_labels_categorical[:male_training_samples], 
                      validation_data=(np.expand_dims(male_data[male_validation_samples:], -1), 
                      male_labels_categorical[male_validation_samples:]),
                      class_weight=male_class_weights, epochs=400, shuffle=True)  # class_weight=class_weights, 

In [ ]:
### loss plots using model A
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(1, len(loss) + 1)

plt.plot(epochs, loss, 'ro', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.show()

In [ ]:
### accuracy plots using model A
plt.clf()                                                

acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

plt.plot(epochs, acc, 'ro', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

plt.show()

In [ ]:
j = 1
print ("CNN {0:d}: Epochs={1:d}, Train accuracy={2:.5f}, Validation accuracy={3:.5f}".format(
        j + 1, 100, max(history.history['accuracy']), max(history.history['val_accuracy'])))
# predictions = model_A.predict(np.expand_dims(data[test_samples:], -1))
# predictions[0:5]
train_acc = model_C.evaluate(np.expand_dims(male_data[:male_training_samples], -1), male_labels_categorical[:male_training_samples], verbose=0)
test_acc = model_C.evaluate(np.expand_dims(male_data[male_test_samples:], -1),
                                male_labels_categorical[male_test_samples:], verbose=0)
val_acc = model_C.evaluate(np.expand_dims(male_data[male_validation_samples:], -1), male_labels_categorical[male_validation_samples:], verbose=0)
print ("accuracy printing")
print (train_acc)
print (test_acc)
print (val_acc)

In [ ]:
### train using model A
com_data = np.r_[male_data[:male_training_samples], female_data[:female_training_samples]]
com_labels = np.r_[male_labels_categorical[:male_training_samples], female_labels_categorical[:female_training_samples]]
com_labels.shape
com_data.shape

# com_labels_categorical = to_categorical(com_labels)
# com_labels_categorical.shape

model_check = female_create_model_LSTM()
history = model_check.fit(np.expand_dims(com_data[:male_training_samples+female_training_samples],-1), 
                      com_labels[:male_training_samples+female_training_samples], 
                      validation_split=0.20, epochs=450)

In [ ]:
### loss plots using model A
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(1, len(loss) + 1)

plt.plot(epochs, loss, 'ro', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.show()

### accuracy plots using model A
plt.clf()                                                

acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

plt.plot(epochs, acc, 'ro', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

plt.show()

In [ ]:
j = 1
print ("CNN {0:d}: Epochs={1:d}, Train accuracy={2:.5f}, Validation accuracy={3:.5f}".format(
        j + 1, 100, max(history.history['accuracy']), max(history.history['val_accuracy'])))
# predictions = model_A.predict(np.expand_dims(data[test_samples:], -1))
# predictions[0:5]
train_acc = model_check.evaluate(np.expand_dims(com_data[:male_training_samples+female_training_samples],-1), 
                      com_labels[:male_training_samples+female_training_samples], verbose=0)
test_acc = model_check.evaluate(np.expand_dims(com_data[:male_training_samples+female_training_samples],-1), 
                      com_labels[:male_training_samples+female_training_samples], verbose=0)
val_acc = model_check.evaluate(np.expand_dims(com_data[:male_training_samples+female_training_samples],-1), 
                      com_labels[:male_training_samples+female_training_samples], verbose=0)
print ("accuracy printing")
print (train_acc)
print (test_acc)
print (val_acc)

In [ ]:
# from keras.models import Model

# mergedOutput = Concatenate()([model_A.output, model_C.output])
# out = Dense(8, activation='softmax')(mergedOutput)
# model = Model([model_A.input, model_C.input], out)
# print(model.summary())
# model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])


final_data = np.r_[male_audio_only_data_array, female_audio_only_data_array, male_cremad_only_data_array, female_cremad_only_data_array, male_audio_only_data_song_array, female_audio_only_data_song_array, male_ravdess_song_data_array, female_ravdess_song_data_array, male_savee_data_array, male_ravdess_speech_data_array, female_ravdess_speech_data_array]
final_labels = np.r_[male_audio_only_labels_array, female_audio_only_labels_song_array, male_cremad_only_labels_array, female_cremad_only_labels_array, male_audio_only_labels_song_array, female_audio_only_labels_array, male_ravdess_song_label_array, female_ravdess_song_label_array, male_savee_label_array, male_ravdess_speech_label_array, female_ravdess_speech_label_array]

final_labels_categorical = to_categorical(final_labels)
final_data.shape
final_labels_categorical.shape

final_number_of_samples = final_data.shape[0]
final_training_samples = int(final_number_of_samples * 0.8)
final_validation_samples = int(final_training_samples * 0.33)
final_test_samples = int(final_number_of_samples-(final_training_samples))
final_class_weights = class_weight.compute_class_weight('balanced', np.unique(final_labels_categorical[final_training_samples]),
                                                  final_labels_categorical[final_training_samples])
print (final_training_samples)
print (final_validation_samples)
print (final_test_samples)

# #### combine data
# com_data = np.r_[male_data[:male_training_samples], female_data[:female_training_samples]]
# com_labels = np.r_[male_labels_categorical[:male_training_samples], female_labels_categorical[:female_training_samples]]
# com_labels.shape
# com_data.shape
# np.expand_dims(com_data[:male_training_samples+female_training_samples],-1)

# # final_model = male_female_pitch()  # male_create_model_LSTM()
# x = (np.expand_dims(male_data[male_training_samples]+female_data[female_training_samples],-1))
# print (x)
# y = (male_labels_categorical[:male_training_samples]+female_labels_categorical[:female_training_samples])
# y1 = without_gender_labels_categorical[without_gender_training_samples]
# print(y)

# history = model.fit([np.expand_dims(male_data[:male_training_samples],-1), np.expand_dims(female_data[:female_training_samples],-1)], 
#                       [male_labels_categorical[:male_training_samples], female_labels_categorical[:female_training_samples]]) 

# [np.expand_dims(male_data[:male_training_samples],-1), np.expand_dims(female_data[:female_training_samples],-1)],y

# history = model.fit([np.expand_dims(com_data[:male_training_samples+female_training_samples],-1), np.expand_dims(com_data[:male_training_samples+female_training_samples],-1)],
#                     com_labels[:male_training_samples+female_training_samples], validation_split = 0.33
#                     epochs=400)

model = male_female_pitch()  # male_create_model_LSTM()
history = model.fit(np.expand_dims(final_data[:final_training_samples],-1), 
                      final_labels_categorical[:final_training_samples], 
                      validation_data=(np.expand_dims(final_data[final_validation_samples:], -1), 
                      final_labels_categorical[final_validation_samples:]),
                      class_weight=final_class_weights, epochs=800, shuffle=True) 

In [ ]:
### loss plots using model A
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(1, len(loss) + 1)

plt.plot(epochs, loss, 'ro', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.show()

### accuracy plots using model A
plt.clf()                                                

acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

plt.plot(epochs, acc, 'ro', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

plt.show()

In [ ]:
j = 1
print ("CNN {0:d}: Epochs={1:d}, Train accuracy={2:.5f}, Validation accuracy={3:.5f}".format(
        j + 1, 100, max(history.history['accuracy']), max(history.history['accuracy'])))
# predictions = model_A.predict(np.expand_dims(data[test_samples:], -1))
# predictions[0:5]
train_acc = model.evaluate(np.expand_dims(final_data[:final_training_samples], -1), final_labels_categorical[:final_training_samples], verbose=0)
test_acc = model.evaluate(np.expand_dims(final_data[final_test_samples:], -1),
                                final_labels_categorical[final_test_samples:], verbose=0)
val_acc = model.evaluate(np.expand_dims(final_data[final_validation_samples:], -1), final_labels_categorical[final_validation_samples:], verbose=0)
print ("accuracy printing")
print (train_acc)
print (test_acc)
print (val_acc)

In [ ]:
### plot a histogram to understand the distribution of the data
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker

import collections

# sizes = collections.Counter(labels)
# print (sizes)

# plt.gca().xaxis.set_major_locator(mticker.MultipleLocator(1))
# x = np.arange(8)
# colors = ['red', 'blue', 'green', 'yellow', 'purple', 'orange', 'grey', 'brown']
# plt.hist(labels, color = colors, normed=False, bins=20) 
# plt.xticks(x, ['neutral','calm','happy', 'sad', 'angry', 'fear', 'disgust', 'surprised'])
# # plt.hist(labels)
# plt.show()

# fig, ax = plt.subplots()

# N, bins, patches = ax.hist(labels, edgecolor='white', linewidth=1, normed=True, bins=20)

# colors = ['red', 'blue', 'green', 'yellow', 'purple', 'orange', 'grey', 'brown']
# for i in range(0, 1):
#     patches[i].set_facecolor('r')
# for i in range(1, 3):
#     patches[i].set_facecolor('yellow')
# for i in range(3, 4):
#     patches[i].set_facecolor('g')

# plt.xticks(x, ['neutral','calm','happy', 'sad', 'angry', 'fear', 'disgust', 'surprised'])
# plt.show()


# 01 = neutral, 02 = calm, 03 = happy, 04 = sad, 05 = angry, 06 = fearful, 07 = disgust, 08 = surprised)
# Counter({4: 1307, 2: 1307, 5: 1307, 3: 1307, 6: 1019, 0: 983, 1: 576, 7: 348})
# 0: 12% 1: 7% 2: 16% 3: 16% 4: 16% 5: 16% 6: 13% 7: 4%
female_labels = [r'happy(20%):1,152', r'sad(20%):1,152', r'angry(20%):1,152', r'fear(20%):1,152', r'disgust(15%):888', r'surprised(5%):288']  # r'neutral', r'calm', 
male_labels = [r'happy(20%):1,307', r'sad(20%):1,307', r'angry(20%):1,307', r'fear(20%):1,307', r'disgust(15%):1,019', r'surprised(5%):348']
colors = ['red', 'blue', 'green', 'yellow', 'purple', 'cyan']
female_sizes = [1152, 1152, 1152, 1152, 888, 288]
male_sizes = [1307, 1307, 1307, 1307, 1019, 348]  # 983, 576, 
patches, texts = plt.pie(female_sizes, colors=colors, startangle=90)
plt.legend(patches, female_labels, loc="best")
# Set aspect ratio to be equal so that pie is drawn as a circle.
plt.axis('equal')
plt.tight_layout()
plt.show()

patches, texts = plt.pie(male_sizes, colors=colors, startangle=90)
plt.legend(patches, male_labels, loc="best")
# Set aspect ratio to be equal so that pie is drawn as a circle.
plt.axis('equal')
plt.tight_layout()
plt.show()

gender_labels = [r'male gender 53.27%(6595)', r'female gender 46.73%(5784)']
gender_sizes = [(6595/12379)*100, (5784/12379)*100]
gender_colors = ['red', 'blue']
patches, texts = plt.pie(gender_sizes, colors=gender_colors, startangle=90)
plt.legend(patches, gender_labels, loc="best")
# Set aspect ratio to be equal so that pie is drawn as a circle.
plt.axis('equal')
plt.tight_layout()
plt.show()